# Hypercube model

Let's start by playing with a hypercube model.
In this model we imagine that every person has a number of neighbours that they could potentially get/give covid-19 from/to.

If we imagined a matrix, then each person would have 8 neighbours.
In a cube tensor, each person would have $8 + 9 + 9 = 26$ neighbours.
In an n-dimensional tensor, each person has $3^n - 1$ neighbours.

Let's begin with an easy tensor. 0 represents someone who is uninfected, 1 represents someone who is infected, and NA represents either someone who doesn't exist or someone who is no longer at risk (dead or immune).

In [18]:
N = 4
total_people = 1000

tensor = zeros(Union{Missing, Int8}, (Int(ceil(total_people^(1/N))) for i in 1:N)...)
total_neighbours = 3^N - 1

using StatsBase
locs = sample(1:((Int(ceil(total_people^(1/N))))^N),((Int(ceil(total_people^(1/N))))^N - total_people), replace=false)
tensor[locs] .= missing;

We begin by making a few `n0` people unwell.

In [19]:
n0 = 5
locs = sample(1:total_people, n0)
tensor[locs] .= 1;
sum(skipmissing(tensor)) ##should be 5

5

First we iterate over each of the people who are unwell.
The probability of them dying in that step is `mortrate`.
The probability of them becoming well in that step is `healrate`.

Then each step we iterate over the neighbours of people with illness.
The average number of people who become infected by someone who is infected is `r0`.
This must be done in the average time of being infected, i.e. 1/(`mortrate` + `healrate`).
Therefore the average number of infections per step is `r0` * (`mortrate` + `healrate`).

The probability of each of the neighbours becoming 1 is `r0` * (`mortrate` + `healrate`) * 1 / `total_neighbours`.
If someone has two neighbours they will be iterated over twice.

Then we can sum the array to know the number infected.

In [ ]:
function inflect(mortrate, healrate, r0, N)
    function infect(array, cartind):
        selected = array[cartind[i]+1:cartind[i]-1 for i in cartind]
        
    return infect
    end
end

        




In [24]:
function find_indices(pos, len)
    start = pos - 1
    if pos == 1
        start = 1
    end
    last = pos + 1
    if pos == len
        last = len
    end
    return(start:last)
end

find_indices (generic function with 1 method)

In [28]:
function get_neighbours(array, cartind)
    dims = ndims(array)
    len = size(array)[1]
    tpl = Tuple(cartind)

    array[(find_indices(i, len) for i in tpl)...]
    
end

get_neighbours (generic function with 1 method)

In [29]:
get_neighbours(tensor, findall(coalesce.(tensor .== 1, false))[1])

3×3×3×2 Array{Union{Missing, Int8},4}:
[:, :, 1, 1] =
 0          missing   missing
 0         0         0       
  missing  0         0       

[:, :, 2, 1] =
 0         0  0
 0         1  0
  missing  0  0

[:, :, 3, 1] =
 0  0          missing
 0   missing  0       
 0  0         0       

[:, :, 1, 2] =
 missing  0         missing
 missing  0         missing
 missing   missing  missing

[:, :, 2, 2] =
 0  0         0
 0   missing  0
 0  0         0

[:, :, 3, 2] =
 0         0   missing
  missing  0  0       
 0         0  0       

In [97]:


[ for j,k in [-1, 0, 1] for i in 1:4]

LoadError: syntax: invalid iteration specification

In [89]:
for i in findall(coalesce.(tensor .== 1, false))
    println((Tuple(i)))
    for j in Tuple(i)
        println(j)
    end
end


(5, 3, 2, 1)
5
3
2
1
(4, 4, 3, 1)
4
4
3
1
(6, 6, 1, 2)
6
6
1
2
(1, 2, 5, 3)
1
2
5
3
(2, 1, 1, 5)
2
1
1
5


In [20]:
findall(coalesce.(tensor .== 1, false))

5-element Array{CartesianIndex{4},1}:
 CartesianIndex(3, 3, 3, 1)
 CartesianIndex(5, 4, 1, 2)
 CartesianIndex(6, 1, 1, 3)
 CartesianIndex(5, 1, 5, 3)
 CartesianIndex(1, 6, 4, 4)

In [ ]:
for i in findall(coalesce.(tensor .==1, false))
    Tuple(i)

In [31]:
tensor[2:4,2:4,2:4,1:2]

3×3×3×2 Array{Union{Missing, Int8},4}:
[:, :, 1, 1] =
 0  0   missing
 0  0  0       
 0  0  0       

[:, :, 2, 1] =
 0          missing  0
 0         1         0
  missing   missing  0

[:, :, 3, 1] =
 0   missing  0       
 0  0          missing
 0   missing  0       

[:, :, 1, 2] =
  missing  0  0       
 0         0   missing
 0         0  0       

[:, :, 2, 2] =
 0         0         0
  missing   missing  0
 0         0         0

[:, :, 3, 2] =
 0  0         0
 0   missing  0
 0  0         0